In [1]:
from getpass import getpass
from mysql.connector import connect
import pandas as pd

In [2]:
# Create connection object
try:
    connection = connect(
        host="localhost",
        user=input("Enter username: "),
        password=getpass("Enter password: "),
        database="sakila"
    )
    print('Connected!')
except:
    print('Unable to connect!')

Enter username:  root
Enter password:  ···········


Connected!


In [3]:
# Create cursor object on connection object
cursor = connection.cursor()
# The cursor is used to run queries and fetch results

In [4]:
def run_query(query):
    '''Pass query string'''
    cursor.execute(query)
    result = cursor.fetchall()
    df = pd.DataFrame(result)
    return df

## What are the tables in the sakila database?

In [5]:
run_query('show tables')

,0
0,actor
1,actor_info
2,address
3,category
4,city
5,country
6,customer
7,customer_list
8,film
9,film_actor


## Show the features of the Customer and  Rental tables

In [6]:
run_query('desc customer')

,0,1,2,3,4,5
0,customer_id,b'smallint unsigned',NO,PRI,None,auto_increment
1,store_id,b'tinyint unsigned',NO,MUL,None,
2,first_name,b'varchar(45)',NO,,None,
3,last_name,b'varchar(45)',NO,MUL,None,
4,email,b'varchar(50)',YES,,None,
5,address_id,b'smallint unsigned',NO,MUL,None,
6,active,b'tinyint(1)',NO,,b'1',
7,create_date,b'datetime',NO,,None,
8,last_update,b'timestamp',YES,,b'CURRENT_TIMESTAMP',DEFAULT_GENERATED on update CURRENT_TIMESTAMP


In [7]:
run_query('desc rental')

,0,1,2,3,4,5
0,rental_id,b'int',NO,PRI,None,auto_increment
1,rental_date,b'datetime',NO,MUL,None,
2,inventory_id,b'mediumint unsigned',NO,MUL,None,
3,customer_id,b'smallint unsigned',NO,MUL,None,
4,return_date,b'datetime',YES,,None,
5,staff_id,b'tinyint unsigned',NO,MUL,None,
6,last_update,b'timestamp',NO,,b'CURRENT_TIMESTAMP',DEFAULT_GENERATED on update CURRENT_TIMESTAMP


## Which is the longer renting period?

In [8]:
run_query('select max(datediff(return_date, rental_date)) from rental')

,0
0,10


# What is the frequency of each renting period?

In [9]:
run_query('select diff, count(*)\
from (select customer_id, datediff(return_date, rental_date) as diff from rental) as der group by 1 order by 2 desc')

,0,1
0,7.0,1821
1,2.0,1795
2,6.0,1783
3,8.0,1762
4,5.0,1761
5,3.0,1714
6,9.0,1691
7,4.0,1681
8,1.0,1644
9,NaN,183


## What is the average rental period?

In [10]:
run_query('select concat(round(avg(datediff(return_date, rental_date)))," ", "days") from rental')

,0
0,5 days


## Who are the people that rented films for a longer period?

In [11]:
run_query('select distinct(c.customer_id), c.first_name, c.last_name\
          from customer as c inner join rental as r\
          using(customer_id)\
         where customer_id in (select customer_id from rental where datediff(return_date, rental_date) = \
         (select max(datediff(return_date, rental_date))from rental))')

,0,1,2
0,1,MARY,SMITH
1,11,LISA,ANDERSON
2,13,KAREN,JACKSON
3,21,MICHELLE,CLARK
4,23,SARAH,LEWIS
...,...,...,...
94,565,JAIME,NETTLES
95,583,MARSHALL,THORN
96,591,KENT,ARSENAULT
97,592,TERRANCE,ROUSH


## How many people have not returned a rented film?

In [12]:
run_query('select count(*) \
from rental \
where (rental_date is not null) and (return_date is null)')

,0
0,183


## How many films were rented in 2005 and 2006?

In [13]:
run_query('select extract(year from rental_date), count(*)\
 from rental\
 group by extract(year from rental_date)')

,0,1
0,2005,15862
1,2006,182


## Describe Payment table

In [14]:
run_query('desc payment')

,0,1,2,3,4,5
0,payment_id,b'smallint unsigned',NO,PRI,None,auto_increment
1,customer_id,b'smallint unsigned',NO,MUL,None,
2,staff_id,b'tinyint unsigned',NO,MUL,None,
3,rental_id,b'int',YES,MUL,None,
4,amount,"b'decimal(5,2)'",NO,,None,
5,payment_date,b'datetime',NO,,None,
6,last_update,b'timestamp',YES,,b'CURRENT_TIMESTAMP',DEFAULT_GENERATED on update CURRENT_TIMESTAMP


## What is the max, min, and average amount paid by customers?

In [15]:
run_query('select max(amount) as Max, min(amount) as Min, round(avg(amount), 2) as Mean from payment')

,0,1,2
0,11.99,0.00,4.20


## What is the average amount spent by each customer?

In [16]:
run_query('select c.customer_id, c.first_name, c.last_name, round(avg(p.amount),2)\
from payment as p inner join customer as c using(customer_id)\
group by c.customer_id')

,0,1,2,3
0,1,MARY,SMITH,3.71
1,2,PATRICIA,JOHNSON,4.77
2,3,LINDA,WILLIAMS,5.22
3,4,BARBARA,JONES,3.72
4,5,ELIZABETH,BROWN,3.81
...,...,...,...,...
594,595,TERRENCE,GUNDERSON,3.92
595,596,ENRIQUE,FORSYTHE,3.45
596,597,FREDDIE,DUGGAN,3.99
597,598,WADE,DELVALLE,3.81


## What is the number of payments made by each customer, and who are those people who have made at least 40 payments?

In [17]:
run_query('select customer_id, count(*) from payment group by customer_id order by 2 desc')

,0,1
0,148,46
1,526,45
2,144,42
3,236,42
4,75,41
...,...,...
594,248,15
595,61,14
596,110,14
597,281,14


In [18]:
run_query('select c.customer_id, c.first_name, c.last_name, count(*)\
from payment as p inner join customer as c \
on c.customer_id=p.customer_id \
group by p.customer_id having count(*) >= 40 order by 4 desc')

,0,1,2,3
0,148,ELEANOR,HUNT,46
1,526,KARL,SEAL,45
2,144,CLARA,SHAW,42
3,236,MARCIA,DEAN,42
4,75,TAMMY,SANDERS,41
5,197,SUE,PETERS,40
6,469,WESLEY,BULL,40


## What is the total amount spend by each costumer?

In [19]:
run_query('select customer_id, sum(amount) from payment group by customer_id order by 2 desc')

,0,1
0,526,221.55
1,148,216.54
2,144,195.58
3,137,194.61
4,178,194.61
...,...,...
594,97,58.82
595,395,57.81
596,318,52.88
597,281,50.86


In [20]:
#diconnect from server
cursor.close()
connection.close()